In [4]:
from bs4 import BeautifulSoup
from splinter import Browser
import requests
import pymongo
import pandas as pd 
import time
import datetime

In [5]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [6]:
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
html = browser.html 


In [7]:
soup1 = BeautifulSoup(html, 'html.parser')


In [15]:
art_title = soup1.select_one('ul.item_list li.slide')
art_title

<li class="slide"><div class="image_and_description_container"><a href="/news/8447/watch-nasa-build-its-next-mars-rover/" target="_self"><div class="rollover_description"><div class="rollover_description_inner">A newly installed webcam offers the public a live, bird's-eye view of NASA's Mars 2020 rover as it takes shape at NASA's Jet Propulsion Laboratory. </div><div class="overlay_arrow"><img alt="More" src="/assets/overlay-arrow.png"/></div></div><div class="list_image"><img alt='The View from the "Seeing 2020" Webcam' src="/system/news_items/list_view_images/8447_seeing_2020-th.jpg"/></div><div class="bottom_gradient"><div><h3>Watch NASA Build Its Next Mars Rover</h3></div></div></a><div class="list_text"><div class="list_date">June  6, 2019</div><div class="content_title"><a href="/news/8447/watch-nasa-build-its-next-mars-rover/" target="_self">Watch NASA Build Its Next Mars Rover</a></div><div class="article_teaser_body">A newly installed webcam offers the public a live, bird's-ey

In [16]:
news_title = art_title.find("div", class_='content_title').get_text()

'Watch NASA Build Its Next Mars Rover'

In [20]:
art_teaser = art_title.find("div", class_='article_teaser_body')
news_p = art_teaser.get_text()
news_p

"A newly installed webcam offers the public a live, bird's-eye view of NASA's Mars 2020 rover as it takes shape at NASA's Jet Propulsion Laboratory. "

In [24]:
imageurl = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(imageurl)

In [26]:

#Python finds the "full_image" id on the page
fullimage = browser.find_by_id("full_image")

In [27]:
fullimage.click()

In [28]:
imagehome = browser.find_link_by_partial_text('more info')
imagehome.click()

In [29]:
html2 = browser.html 
soup2 = BeautifulSoup(html2, "html.parser")

In [34]:
plus_url = soup2.select_one('figure.lede a img').get('src')

In [39]:
featured_image_url = "https://www.jpl.nasa.gov"+plus_url 
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA14400_hires.jpg'

In [40]:
twitmars = "https://twitter.com/marswxreport?lang=en"
browser.visit(twitmars)
html3 = browser.html




tweetsoup = BeautifulSoup(html3, "html.parser")

In [42]:
mars_weather_tweet = tweetsoup.find('div', attrs={"class": "tweet", "data-name": "Mars Weather"})
mars_weather_tweet

<div class="tweet js-stream-tweet js-actionable-tweet js-profile-popup-actionable dismissible-content original-tweet js-original-tweet has-cards has-content " data-conversation-id="1137311111185948672" data-disclosure-type="" data-follows-you="false" data-has-cards="true" data-item-id="1137311111185948672" data-name="Mars Weather" data-permalink-path="/MarsWxReport/status/1137311111185948672" data-reply-to-users-json='[{"id_str":"786939553","screen_name":"MarsWxReport","name":"Mars Weather","emojified_name":{"text":"Mars Weather","emojified_text_as_html":"Mars Weather"}}]' data-screen-name="MarsWxReport" data-tweet-id="1137311111185948672" data-tweet-nonce="1137311111185948672-1b0a026f-b34e-4e2d-9fef-592d833b4ef0" data-tweet-stat-initialized="true" data-user-id="786939553" data-you-block="false" data-you-follow="false">
<div class="context">
</div>
<div class="content">
<div class="stream-item-header">
<a class="account-group js-account-group js-action-profile js-user-profile-link js-n

In [44]:
mars_weather = mars_weather_tweet.find('p', 'tweet-text').get_text()
mars_weather

'InSight sol 188 (2019-06-07) low -102.5ºC (-152.6ºF) high -21.9ºC (-7.4ºF)\nwinds from the SSE at 4.8 m/s (10.8 mph) gusting to 15.6 m/s (35.0 mph)\npressure at 7.60 hPapic.twitter.com/ocUTA1rgaU'

In [48]:
df = pd.read_html('http://space-facts.com/mars/')[0]
df.columns=['description', 'value']
df.set_index('description', inplace=True)
df

# use df.to_html()

,value
description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [49]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)
hemisphere_image_urls = []
# First, get a list of all of the hemispheres
links = browser.find_by_css("a.product-item h3")
# Next, loop through those links, click the link, find the sample anchor, return the href
for i in range(len(links)):
    hemisphere = {}
    
    # We have to find the elements on each loop to avoid a stale element exception
    browser.find_by_css("a.product-item h3")[i].click()
    
    # Next, we find the Sample image anchor tag and extract the href
    sample_elem = browser.find_link_by_text('Sample').first
    hemisphere['img_url'] = sample_elem['href']
    
    # Get Hemisphere title
    hemisphere['title'] = browser.find_by_css("h2.title").text
    
    # Append hemisphere object to list
    hemisphere_image_urls.append(hemisphere)
    
    # Finally, we navigate backwards
    browser.back()


In [57]:
hemisphere_image_urls

[{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [55]:
hemisphere_image_urls[0]["img_url"]

'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'

In [ ]:
# for x in range(50):
#     # HTML object
#     html = browser.html
#     # Parse HTML with Beautiful Soup
#     soup = BeautifulSoup(html, 'html.parser')
#     # Retrieve all elements that contain book information
#     content_title = soup.find_all('title', class_='content_title')

#     # Iterate through each book
#     for title in content_title:
#         # Use Beautiful Soup's find() method to navigate and retrieve attributes
#         h3 = article.find('_self')
#     #print(title)

# #     # Click the 'Next' button on each page
# #     try:
# #         browser.click_link_by_partial_text('next')
          
# #     except:
# #         print("Scraping Complete")


In [ ]:
# print(soup)

In [ ]:
# url2 = "https://www.cracked.com/"

In [ ]:
# browser.visit(url2)
# html = browser.html


In [ ]:
# for x in range(50):
#     # HTML object
#     html = browser.html
#     # Parse HTML with Beautiful Soup
#     soup2 = BeautifulSoup(html, 'html.parser')
# #     # Retrieve all elements that contain book information
# #     content_title = soup.find_all('title', class_='content_title')



In [ ]:
# print(soup2)

In [ ]:
# article_title = soup.find("caption", class_="title")

In [ ]:
# print(article_title)